# LU Factorization

In [1]:
# Library:
# jupyterlab           3.3.1 
# numpy                1.22.3 
# sympy                1.10

import numpy as np  # Library for mathematical operation
import sympy as sy  # Library for printing matrix equation

## Metoda Doolittle’a
$$ A = LU $$

$$
{\begin{bmatrix}a_{11}&a_{12}&\cdots &a_{1n}\\a_{21}&a_{22}&\cdots &a_{2n}\\\vdots &\vdots &\ddots &\vdots \\a_{n1}&a_{n2}&\cdots &a_{nn}\end{bmatrix}}={\begin{bmatrix}1&0&\cdots &0\\l_{21}&1&\cdots &0\\\vdots &\vdots &\ddots &0\\l_{n1}&l_{n2}&\cdots &1\end{bmatrix}}\cdot {\begin{bmatrix}u_{11}&u_{12}&\cdots &u_{1n}\\0&u_{22}&\cdots &u_{2n}\\\vdots &\vdots &\ddots &\vdots \\0&0&\cdots &u_{nn}\end{bmatrix}}
$$

$ u_{ij}=a_{ij}-\sum _{k=1}^{i-1}l_{ik}u_{kj}$ dla $j\in \{i,\ i+1,\ldots ,\ n\}$

$ l_{ji}={\frac {1}{u_{ii}}}\left(a_{ji}-\sum _{k=1}^{i-1}l_{jk}u_{ki}\right)$ dla $j\in \{i+1,\ i+2,\ldots ,\ n\}$

In [225]:
# Class to do factorization
class LUFactorization:
    @staticmethod
    def DollitleMethod( A ):
        rows, cols = A.shape                      # Get matrix shape rows and cols
        L = np.matrix(np.diag( np.ones( rows ) )) # Create matrix L (1 on diagonal)
        U = np.matrix(np.zeros( A.shape ))        # Create zeros matrix U 
        
        L[:, 0] = A[:, 0] / A[0,0]   # First column in L matrix is first column A / A(0,0)
        U[0, :] = A[0, :]            # First row in U matrix is first column of A

        for i in range( 1, rows ):                # For all rows in matrix
            for j in range( i, cols):             # For all columns in i row
                U[i,j] = A[i,j]-L[i,0:i]*U[0:i,j] # U(i,j) = A(i,j)-sum(L(i,k)*U(k,j)), where k[1,i-1]
                
            
            for j in range( i+1, rows ):                   # For all rows in i column
                L[j,i] = (A[j,i]-L[j,0:i]*U[0:i,i])/U[i,i] # L(j,i) = A(j,i)-sum(L(j,k)*U(k,i)), where k[1,i-1]
    
        return (L, U)

In [321]:
# Class to do substitution
class Substitution:
    @staticmethod
    def Forward( L, b ):
        rows, cols = b.shape
        x = np.matrix(np.zeros(b.shape))
        x[0,0] = b[0,0]/L[0,0]
        for i in range(1, rows):
            x[i,0] = (L[i,0:i]*b[0:i,0])/L[i,i]
        return x
    
    @staticmethod
    def Backward( U, b ):
        rows, cols = U.shape
        x = np.matrix(np.zeros(b.shape), dtype=float)
        x[rows-1, 0] = b[rows-1, 0]/U[rows-1, cols-1]
        for i in range(rows-2, -1, -1):
            x[i,0] = (U[i,i+1:cols]*x[i+1:rows,0])/U[i,i]
        return x
        

In [322]:
# Matrix to LU Factorization

A = np.matrix( [ [1, 2, 3, 4], [-1, 1, 2, 1], [0, 2, 1, 3], [0, 0, 1, 1] ], dtype=float) 
#A = np.matrix( [ [5,3,2],[1,2,0],[3,0,4] ], dtype=float )

In [323]:
# Display result A=LU
L, U = LUFactorization.DollitleMethod(A)
A_sy = sy.Matrix(A)
L_sy = sy.Matrix(L.round(3))
U_sy = sy.Matrix(U.round(3))

sy.Eq( A_sy, sy.MatMul(L_sy, U_sy, evaluate=False), evaluate=False )

Eq(Matrix([
[ 1.0, 2.0, 3.0, 4.0],
[-1.0, 1.0, 2.0, 1.0],
[   0, 2.0, 1.0, 3.0],
[   0,   0, 1.0, 1.0]]), Matrix([
[ 1.0,     0,      0,   0],
[-1.0,   1.0,      0,   0],
[   0, 0.667,    1.0,   0],
[   0,     0, -0.429, 1.0]])*Matrix([
[1.0, 2.0,    3.0,    4.0],
[  0, 3.0,    5.0,    5.0],
[  0,   0, -2.333, -0.333],
[  0,   0,      0,  0.857]]))

In [324]:
# Reverse LU=A'
LU = L*U
LU_sy = sy.Matrix(LU.round(3))
sy.Eq( sy.MatMul(L_sy, U_sy, evaluate=False), LU_sy,  evaluate=False )

Eq(Matrix([
[ 1.0,     0,      0,   0],
[-1.0,   1.0,      0,   0],
[   0, 0.667,    1.0,   0],
[   0,     0, -0.429, 1.0]])*Matrix([
[1.0, 2.0,    3.0,    4.0],
[  0, 3.0,    5.0,    5.0],
[  0,   0, -2.333, -0.333],
[  0,   0,      0,  0.857]]), Matrix([
[ 1.0, 2.0, 3.0, 4.0],
[-1.0, 1.0, 2.0, 1.0],
[   0, 2.0, 1.0, 3.0],
[   0,   0, 1.0, 1.0]]))